In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from pathlib import Path

In [ ]:
def infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma):
    for i in range(iterations):
        S, I, R = S - beta * ((S * I) / N), I + beta * ((S * I) / N) - gamma * I, R + gamma * I 
        S_comp.append(S)
        I_comp.append(I)
        R_comp.append(R)

In [ ]:
def SIR_mse_calc(N,contamine,susceptible,recovered):
    iterations = len(contamine)
    mse = 1000000000000
    betarange = np.linspace(0,200,201)
    gammarange = np.linspace(0,100,101)
    for pas_beta in betarange:
        beta = pas_beta/100
        for pas_gamma in gammarange:
            gamma = pas_gamma/100
            S = N - 1
            I = 1
            R = 0
            S_comp = [] # infected compartment
            I_comp = [] # susceptible compartment
            R_comp = [] # recovered compartment
            infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
            newmsi = np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean() + np.square(np.subtract(susceptible.values.tolist(),S_comp)).mean() + np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean()
            if(newmsi < mse):
                mse = newmsi
                betavalue = beta
                gammavalue = gamma
    return [mse,betavalue,gammavalue]

## Densité population 1/2

### 5000 individus

In [ ]:
path5k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population5000"
path5k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population5000"

In [ ]:
temps = pd.read_csv(path5k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path5k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 5000
temps_5000_mouvements = pd.read_csv(path5k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine5k1 = pd.read_csv(path5k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k1 = pd.read_csv(path5k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine5k2 = pd.read_csv(path5k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k2 = pd.read_csv(path5k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 1.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine5k1, label='I_5000(t)')
I_MIX, = plt.plot(contamine5k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered5k1, label='R_5000(t)')
R_MIX, = plt.plot(recovered5k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path5k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path5k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path5k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path5k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/2, 5000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/2, 5000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible5k1 = (N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine5k1,susceptible5k1,recovered5k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible5k2 = (N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine5k2,susceptible5k2,recovered5k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine5k1) < len(contamine5k2)):
    contamine5k2 = contamine5k2[:len(contamine5k1)]
    recovered5k2 = recovered5k2[:len(recovered5k1)]
    susceptible5k2 = susceptible5k2[:len(susceptible5k1)]
else:
    contamine5k1 = contamine5k1[:len(contamine5k2)]
    recovered5k1 = recovered5k1[:len(recovered5k2)]
    susceptible5k1 = susceptible5k1[:len(susceptible5k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine5k1['Nombre de contaminés'].values.tolist(),contamine5k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered5k1['Recovered'].values.tolist(),recovered5k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible5k1.values.tolist(),susceptible5k2.values.tolist())).mean())

### 20000 individus

In [ ]:
path20k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population20000"
path20k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population20000"

In [ ]:
temps = pd.read_csv(path20k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path20k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 20000
temps_5000_mouvements = pd.read_csv(path20k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine20k1 = pd.read_csv(path20k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k1 = pd.read_csv(path20k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine20k2 = pd.read_csv(path20k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k2 = pd.read_csv(path20k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 125

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
SIR_I, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine20k1, label='I_20000(t)')
I_MIX, = plt.plot(contamine20k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered20k1, label='R_20000(t)')
R_MIX, = plt.plot(recovered20k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered']), label='I_20000(t)')
S_MIX, = plt.plot((N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered']), label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path20k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path20k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path20k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path20k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/8, 20000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/8, 20000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible20k1 = (N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine20k1,susceptible20k1,recovered20k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible20k2 = (N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine20k2,susceptible20k2,recovered20k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine20k1) < len(contamine20k2)):
    contamine20k2 = contamine20k2[:len(contamine20k1)]
    recovered20k2 = recovered20k2[:len(recovered20k1)]
    susceptible20k2 = susceptible20k2[:len(susceptible20k1)]
else:
    contamine20k1 = contamine20k1[:len(contamine20k2)]
    recovered20k1 = recovered20k1[:len(recovered20k2)]
    susceptible20k1 = susceptible20k1[:len(susceptible20k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine20k1['Nombre de contaminés'].values.tolist(),contamine20k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered20k1['Recovered'].values.tolist(),recovered20k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible20k1.values.tolist(),susceptible20k2.values.tolist())).mean())

### 50000 individus

In [ ]:
path50k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population50000"
path50k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population50000"

In [ ]:
temps = pd.read_csv(path50k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path50k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 50000
temps_5000_mouvements = pd.read_csv(path50k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine50k1 = pd.read_csv(path50k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k1 = pd.read_csv(path50k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine50k2 = pd.read_csv(path50k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k2 = pd.read_csv(path50k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine50k1, label='I_50000(t)')
I_MIX, = plt.plot(contamine50k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered50k1, label='R_50000(t)')
R_MIX, = plt.plot(recovered50k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path50k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path50k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path50k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path50k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/2, 50000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/2, 50000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible50k1 = (N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine50k1,susceptible50k1,recovered50k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible50k2 = (N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine50k2,susceptible50k2,recovered50k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine50k1) < len(contamine50k2)):
    contamine50k2 = contamine50k2[:len(contamine50k1)]
    recovered50k2 = recovered50k2[:len(recovered50k1)]
    susceptible50k2 = susceptible50k2[:len(susceptible50k1)]
else:
    contamine50k1 = contamine50k1[:len(contamine50k2)]
    recovered50k1 = recovered50k1[:len(recovered50k2)]
    susceptible50k1 = susceptible50k1[:len(susceptible50k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine50k1['Nombre de contaminés'].values.tolist(),contamine50k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered50k1['Recovered'].values.tolist(),recovered50k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible50k1.values.tolist(),susceptible50k2.values.tolist())).mean())

### 100000 individus

In [ ]:
path100k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population100000"
path100k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population100000"

In [ ]:
temps = pd.read_csv(path100k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path100k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 100000
temps_5000_mouvements = pd.read_csv(path100k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine100k1 = pd.read_csv(path100k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k1 = pd.read_csv(path100k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine100k2 = pd.read_csv(path100k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k2 = pd.read_csv(path100k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine100k1, label='I_100000(t)')
I_MIX, = plt.plot(contamine100k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered100k1, label='R_100000(t)')
R_MIX, = plt.plot(recovered100k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path100k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path100k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path100k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path100k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/2, 100000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/2, 100000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible100k1 = (N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine100k1,susceptible100k1,recovered100k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible100k2 = (N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine100k2,susceptible100k2,recovered100k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine100k1) < len(contamine100k2)):
    contamine2 = contamine100k2[:len(contamine100k1)]
    recovered2 = recovered100k2[:len(recovered100k1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine100k1 = contamine100k1[:len(contamine100k2)]
    recovered100k1 = recovered100k1[:len(recovered100k2)]
    susceptible100k1 = susceptible100k1[:len(susceptible100k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine100k1['Nombre de contaminés'].values.tolist(),contamine100k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered100k1['Recovered'].values.tolist(),recovered100k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible100k1.values.tolist(),susceptible100k2.values.tolist())).mean())

### 5000 mouvements comparaisons

In [ ]:
iterations = 100

figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_5000, = plt.plot(contamine5k1/5000, label='I_5000(t)')
I_20000, = plt.plot(contamine20k1/20000, label='I_20000(t)')
I_50000, = plt.plot(contamine50k1/50000, label='I_50000(t)')
I_100000, = plt.plot(contamine100k1/100000, label='I_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Infected normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[I_5000,I_20000,I_50000,I_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_5000, = plt.plot(recovered5k1/5000, label='R_5000(t)')
R_20000, = plt.plot(recovered20k1/20000, label='R_20000(t)')
R_50000, = plt.plot(recovered50k1/50000, label='R_50000(t)')
R_100000, = plt.plot(recovered100k1/100000, label='R_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Recovered normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[R_5000,R_20000,R_50000,R_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_5000, = plt.plot((5000-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])/5000, label='S_5000(t)')
S_20000, = plt.plot((20000-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])/20000, label='S_20000(t)')
S_50000, = plt.plot((50000-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])/50000, label='S_50000(t)')
S_100000, = plt.plot((100000-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])/100000, label='S_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Susceptible normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[S_5000,S_20000,S_50000,S_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

## Densité population 1/4

### 5000 individus

In [ ]:
path5k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population5000"
path5k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population5000"

In [ ]:
temps = pd.read_csv(path5k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path5k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 5000
temps_5000_mouvements = pd.read_csv(path5k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine5k1 = pd.read_csv(path5k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k1 = pd.read_csv(path5k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine5k2 = pd.read_csv(path5k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k2 = pd.read_csv(path5k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine5k1, label='I_5000(t)')
I_MIX, = plt.plot(contamine5k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered5k1, label='R_5000(t)')
R_MIX, = plt.plot(recovered5k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path5k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path5k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path5k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path5k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/4, 5000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/4, 5000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible5k1 = (N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine5k1,susceptible5k1,recovered5k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible5k2 = (N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine5k2,susceptible5k2,recovered5k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine5k1) < len(contamine5k2)):
    contamine5k2 = contamine5k2[:len(contamine5k1)]
    recovered5k2 = recovered5k2[:len(recovered5k1)]
    susceptible5k2 = susceptible5k2[:len(susceptible5k1)]
else:
    contamine5k1 = contamine5k1[:len(contamine5k2)]
    recovered5k1 = recovered5k1[:len(recovered5k2)]
    susceptible5k1 = susceptible5k1[:len(susceptible5k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine5k1['Nombre de contaminés'].values.tolist(),contamine5k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered5k1['Recovered'].values.tolist(),recovered5k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible5k1.values.tolist(),susceptible5k2.values.tolist())).mean())

### 20000 individus

In [ ]:
path20k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population20000"
path20k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population20000"

In [ ]:
temps = pd.read_csv(path20k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path20k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 20000
temps_5000_mouvements = pd.read_csv(path20k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine20k1 = pd.read_csv(path20k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k1 = pd.read_csv(path20k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine20k2 = pd.read_csv(path20k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k2 = pd.read_csv(path20k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine20k1, label='I_20000(t)')
I_MIX, = plt.plot(contamine20k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered20k1, label='R_20000(t)')
R_MIX, = plt.plot(recovered20k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path20k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path20k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path20k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path20k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/4, 20000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/4, 20000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible20k1 = (N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine20k1,susceptible20k1,recovered20k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible20k2 = (N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine20k2,susceptible20k2,recovered20k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine20k1) < len(contamine20k2)):
    contamine20k2 = contamine20k2[:len(contamine20k1)]
    recovered20k2 = recovered20k2[:len(recovered20k1)]
    susceptible20k2 = susceptible20k2[:len(susceptible20k1)]
else:
    contamine20k1 = contamine20k1[:len(contamine20k2)]
    recovered20k1 = recovered20k1[:len(recovered20k2)]
    susceptible20k1 = susceptible20k1[:len(susceptible20k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine20k1['Nombre de contaminés'].values.tolist(),contamine20k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered20k1['Recovered'].values.tolist(),recovered20k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible20k1.values.tolist(),susceptible20k2.values.tolist())).mean())

### 50000 individus

In [ ]:
path50k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population50000"
path50k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population50000"

In [ ]:
temps = pd.read_csv(path50k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path50k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 50000
temps_5000_mouvements = pd.read_csv(path50k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine50k1 = pd.read_csv(path50k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k1 = pd.read_csv(path50k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine50k2 = pd.read_csv(path50k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k2 = pd.read_csv(path50k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine50k1, label='I_50000(t)')
I_MIX, = plt.plot(contamine50k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered50k1, label='R_50000(t)')
R_MIX, = plt.plot(recovered50k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path50k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path50k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path50k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path50k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/4, 50000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/4, 50000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible50k1 = (N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine50k1,susceptible50k1,recovered50k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible50k2 = (N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine50k2,susceptible50k2,recovered50k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine50k1) < len(contamine50k2)):
    contamine50k2 = contamine50k2[:len(contamine50k1)]
    recovered50k2 = recovered50k2[:len(recovered50k1)]
    susceptible50k2 = susceptible50k2[:len(susceptible50k1)]
else:
    contamine50k1 = contamine50k1[:len(contamine50k2)]
    recovered50k1 = recovered50k1[:len(recovered50k2)]
    susceptible50k1 = susceptible50k1[:len(susceptible50k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine50k1['Nombre de contaminés'].values.tolist(),contamine50k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered50k1['Recovered'].values.tolist(),recovered50k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible50k1.values.tolist(),susceptible50k2.values.tolist())).mean())

### 100000 individus

In [ ]:
path100k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population100000"
path100k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population100000"

In [ ]:
temps = pd.read_csv(path100k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path100k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 100000
temps_5000_mouvements = pd.read_csv(path100k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine100k1 = pd.read_csv(path100k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k1 = pd.read_csv(path100k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine100k2 = pd.read_csv(path100k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k2 = pd.read_csv(path100k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine100k1, label='I_100000(t)')
I_MIX, = plt.plot(contamine100k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered100k1, label='R_100000(t)')
R_MIX, = plt.plot(recovered100k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path100k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path100k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path100k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path100k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/4, 100000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/4, 100000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible100k1 = (N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine100k1,susceptible100k1,recovered100k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible100k2 = (N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine100k2,susceptible100k2,recovered100k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine100k1) < len(contamine100k2)):
    contamine100k2 = contamine100k2[:len(contamine100k1)]
    recovered100k2 = recovered100k2[:len(recovered100k1)]
    susceptible100k2 = susceptible100k2[:len(susceptible100k1)]
else:
    contamine100k1 = contamine100k1[:len(contamine100k2)]
    recovered100k1 = recovered100k1[:len(recovered100k2)]
    susceptible100k1 = susceptible100k1[:len(susceptible100k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine100k1['Nombre de contaminés'].values.tolist(),contamine100k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered100k1['Recovered'].values.tolist(),recovered100k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible100k1.values.tolist(),susceptible100k2.values.tolist())).mean())

### 5000 mouvements comparaisons

In [ ]:
iterations = 100

figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_5000, = plt.plot(contamine5k1/5000, label='I_5000(t)')
I_20000, = plt.plot(contamine20k1/20000, label='I_20000(t)')
I_50000, = plt.plot(contamine50k1/50000, label='I_50000(t)')
I_100000, = plt.plot(contamine100k1/100000, label='I_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Infected normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[I_5000,I_20000,I_50000,I_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_5000, = plt.plot(recovered5k1/5000, label='R_5000(t)')
R_20000, = plt.plot(recovered20k1/20000, label='R_20000(t)')
R_50000, = plt.plot(recovered50k1/50000, label='R_50000(t)')
R_100000, = plt.plot(recovered100k1/100000, label='R_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Recovered normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[R_5000,R_20000,R_50000,R_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_5000, = plt.plot((5000-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])/5000, label='S_5000(t)')
S_20000, = plt.plot((20000-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])/20000, label='S_20000(t)')
S_50000, = plt.plot((50000-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])/50000, label='S_50000(t)')
S_100000, = plt.plot((100000-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])/100000, label='S_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Susceptible normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[S_5000,S_20000,S_50000,S_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

## Densité population 1/8

### 5000 individus

In [ ]:
path5k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population5000"
path5k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population5000"

In [ ]:
temps = pd.read_csv(path5k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path5k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 5000
temps_5000_mouvements = pd.read_csv(path5k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine5k1 = pd.read_csv(path5k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k1 = pd.read_csv(path5k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine5k2 = pd.read_csv(path5k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k2 = pd.read_csv(path5k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.39
gamma = 0.06
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine5k1, label='I_5000(t)')
I_MIX, = plt.plot(contamine5k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered5k1, label='R_5000(t)')
R_MIX, = plt.plot(recovered5k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path5k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path5k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path5k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path5k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/8, 5000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/8, 5000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible5k1 = (N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine5k1,susceptible5k1,recovered5k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible5k2 = (N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine5k2,susceptible5k2,recovered5k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine5k1) < len(contamine5k2)):
    contamine5k2 = contamine5k2[:len(contamine5k1)]
    recovered5k2 = recovered5k2[:len(recovered5k1)]
    susceptible5k2 = susceptible5k2[:len(susceptible5k1)]
else:
    contamine5k1 = contamine5k1[:len(contamine5k2)]
    recovered5k1 = recovered5k1[:len(recovered5k2)]
    susceptible5k1 = susceptible5k1[:len(susceptible5k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine5k1['Nombre de contaminés'].values.tolist(),contamine5k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered5k1['Recovered'].values.tolist(),recovered5k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible5k1.values.tolist(),susceptible5k2.values.tolist())).mean())

### 20000 individus

In [ ]:
path20k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population20000"
path20k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population20000"

In [ ]:
temps = pd.read_csv(path20k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 20000
temps_5000_mouvements = pd.read_csv(path20k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine20k1 = pd.read_csv(path20k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k1 = pd.read_csv(path20k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine20k2 = pd.read_csv(path20k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k2 = pd.read_csv(path20k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine20k1, label='I_20000(t)')
I_MIX, = plt.plot(contamine20k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered20k1, label='R_20000(t)')
R_MIX, = plt.plot(recovered20k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path20k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path20k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path20k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path20k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/8, 20000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/8, 20000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible20k1 = (N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine20k1,susceptible20k1,recovered20k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible20k2 = (N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine20k2,susceptible20k2,recovered20k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine20k1) < len(contamine20k2)):
    contamine20k2 = contamine20k2[:len(contamine20k1)]
    recovered20k2 = recovered20k2[:len(recovered20k1)]
    susceptible20k2 = susceptible20k2[:len(susceptible20k1)]
else:
    contamine20k1 = contamine20k1[:len(contamine20k2)]
    recovered20k1 = recovered20k1[:len(recovered20k2)]
    susceptible20k1 = susceptible20k1[:len(susceptible20k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine20k1['Nombre de contaminés'].values.tolist(),contamine20k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered20k1['Recovered'].values.tolist(),recovered20k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible20k1.values.tolist(),susceptible20k2.values.tolist())).mean())

### 50000 individus

In [ ]:
path50k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population50000"
path50k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population50000"

In [ ]:
temps = pd.read_csv(path50k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 50000
temps_5000_mouvements = pd.read_csv(path50k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine50k1 = pd.read_csv(path50k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k1 = pd.read_csv(path50k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine50k2 = pd.read_csv(path50k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k2 = pd.read_csv(path50k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine50k1, label='I_50000(t)')
I_MIX, = plt.plot(contamine50k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered50k1, label='R_50000(t)')
R_MIX, = plt.plot(recovered50k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path50k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path50k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path50k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path50k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/8, 50000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/8, 50000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible50k1 = (N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine50k1,susceptible50k1,recovered50k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible50k2 = (N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine50k2,susceptible50k2,recovered50k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine50k1) < len(contamine50k2)):
    contamine50k2 = contamine50k2[:len(contamine50k1)]
    recovered50k2 = recovered50k2[:len(recovered50k1)]
    susceptible50k2 = susceptible50k2[:len(susceptible50k1)]
else:
    contamine50k1 = contamine50k1[:len(contamine50k2)]
    recovered50k1 = recovered50k1[:len(recovered50k2)]
    susceptible50k1 = susceptible50k1[:len(susceptible50k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine50k1['Nombre de contaminés'].values.tolist(),contamine50k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered50k1['Recovered'].values.tolist(),recovered50k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible50k1.values.tolist(),susceptible50k2.values.tolist())).mean())

### 100000 individus

In [ ]:
path100k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population100000"
path100k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population100000"

In [ ]:
temps = pd.read_csv(path100k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 100000
temps_5000_mouvements = pd.read_csv(path100k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine100k1 = pd.read_csv(path100k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k1 = pd.read_csv(path100k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine100k2 = pd.read_csv(path100k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k2 = pd.read_csv(path100k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine100k1, label='I_100000(t)')
I_MIX, = plt.plot(contamine100k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered100k1, label='R_100000(t)')
R_MIX, = plt.plot(recovered100k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path100k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path100k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path100k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path100k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/8, 100000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/8, 100000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible100k1 = (N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine100k1,susceptible100k1,recovered100k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible100k2 = (N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine100k2,susceptible100k2,recovered100k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine100k1) < len(contamine100k2)):
    contamine100k2 = contamine100k2[:len(contamine100k1)]
    recovered100k2 = recovered100k2[:len(recovered00k1)]
    susceptible100k2 = susceptible100k2[:len(susceptible100k1)]
else:
    contamine100k1 = contamine100k1[:len(contamine100k2)]
    recovered100k1 = recovered100k1[:len(recovered100k2)]
    susceptible100k1 = susceptible100k1[:len(susceptible100k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine100k1['Nombre de contaminés'].values.tolist(),contamine100k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered100k1['Recovered'].values.tolist(),recovered100k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible100k1.values.tolist(),susceptible100k2.values.tolist())).mean())

### 5000 mouvements comparaisons

In [ ]:
iterations = 100

figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_5000, = plt.plot(contamine5k1/5000, label='I_5000(t)')
I_20000, = plt.plot(contamine20k1/20000, label='I_20000(t)')
I_50000, = plt.plot(contamine50k1/50000, label='I_50000(t)')
I_100000, = plt.plot(contamine100k1/100000, label='I_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Infected normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[I_5000,I_20000,I_50000,I_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_5000, = plt.plot(recovered5k1/5000, label='R_5000(t)')
R_20000, = plt.plot(recovered20k1/20000, label='R_20000(t)')
R_50000, = plt.plot(recovered50k1/50000, label='R_50000(t)')
R_100000, = plt.plot(recovered100k1/100000, label='R_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Recovered normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[R_5000,R_20000,R_50000,R_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_5000, = plt.plot((5000-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])/5000, label='S_5000(t)')
S_20000, = plt.plot((20000-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])/20000, label='S_20000(t)')
S_50000, = plt.plot((50000-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])/50000, label='S_50000(t)')
S_100000, = plt.plot((100000-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])/100000, label='S_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Susceptible normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[S_5000,S_20000,S_50000,S_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

## Densité population 1/16

### 5000 individus

In [ ]:
path5k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population5000"
path5k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population5000"

In [ ]:
temps = pd.read_csv(path5k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path5k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 5000
temps_5000_mouvements = pd.read_csv(path5k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine5k1 = pd.read_csv(path5k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k1 = pd.read_csv(path5k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine5k2 = pd.read_csv(path5k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered5k2 = pd.read_csv(path5k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine5k1, label='I_5000(t)')
I_MIX, = plt.plot(contamine5k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered5k1, label='R_5000(t)')
R_MIX, = plt.plot(recovered5k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path5k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path5k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path5k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path5k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/16, 5000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/16, 5000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible5k1 = (N-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine5k1,susceptible5k1,recovered5k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible5k2 = (N-contamine5k2)['Nombre de contaminés'].subtract(recovered5k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine5k2,susceptible5k2,recovered5k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine5k1) < len(contamine5k2)):
    contamine5k2 = contamine5k2[:len(contamine5k1)]
    recovered5k2 = recovered5k2[:len(recovered5k1)]
    susceptible5k2 = susceptible5k2[:len(susceptible5k1)]
else:
    contamine5k1 = contamine1[:len(contamine5k2)]
    recovered5k1 = recovered1[:len(recovered5k2)]
    susceptible5k1 = susceptible1[:len(susceptible5k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine5k1['Nombre de contaminés'].values.tolist(),contamine5k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered5k1['Recovered'].values.tolist(),recovered5k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible5k1.values.tolist(),susceptible5k2.values.tolist())).mean())

### 20000 individus

In [ ]:
path20k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population20000"
path20k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population20000"

In [ ]:
temps = pd.read_csv(path20k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path20k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 20000
temps_5000_mouvements = pd.read_csv(path20k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine20k1 = pd.read_csv(path20k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k1 = pd.read_csv(path20k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine20k2 = pd.read_csv(path20k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered20k2 = pd.read_csv(path20k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine20k1, label='I_20000(t)')
I_MIX, = plt.plot(contamine20k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered20k1, label='R_20000(t)')
R_MIX, = plt.plot(recovered20k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path20k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path20k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path20k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path20k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/16, 20000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/16, 20000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible20k1 = (N-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine20k1,susceptible20k1,recovered20k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible20k2 = (N-contamine20k2)['Nombre de contaminés'].subtract(recovered20k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine20k2,susceptible20k2,recovered20k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine20k1) < len(contamine20k2)):
    contamine20k2 = contamine20k2[:len(contamine20k1)]
    recovered20k2 = recovered20k2[:len(recovered20k1)]
    susceptible20k2 = susceptible20k2[:len(susceptible20k1)]
else:
    contamine20k1 = contamine20k1[:len(contamine20k2)]
    recovered20k1 = recovered20k1[:len(recovered20k2)]
    susceptible20k1 = susceptible20k1[:len(susceptible20k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine20k1['Nombre de contaminés'].values.tolist(),contamine20k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered20k1['Recovered'].values.tolist(),recovered20k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible20k1.values.tolist(),susceptible20k2.values.tolist())).mean())

### 50000 individus

In [ ]:
path50k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population50000"
path50k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population50000"

In [ ]:
temps = pd.read_csv(path50k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path50k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 50000
temps_5000_mouvements = pd.read_csv(path50k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine50k1 = pd.read_csv(path50k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k1 = pd.read_csv(path50k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine50k2 = pd.read_csv(path50k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered50k2 = pd.read_csv(path50k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 200

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine50k1, label='I_50000(t)')
I_MIX, = plt.plot(contamine50k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered50k1, label='R_50000(t)')
R_MIX, = plt.plot(recovered50k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 4

#5000
IX1 = pd.read_csv(path50k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path50k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path50k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path50k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/16, 50000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains')
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/16, 50000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains')
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible50k1 = (N-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine50k1,susceptible50k1,recovered50k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible50k2 = (N-contamine50k2)['Nombre de contaminés'].subtract(recovered50k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine50k2,susceptible50k2,recovered50k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine50k1) < len(contamine50k2)):
    contamine50k2 = contamine50k2[:len(contamine50k1)]
    recovered50k2 = recovered50k2[:len(recovered50k1)]
    susceptible50k2 = susceptible50k2[:len(susceptible50k1)]
else:
    contamine50k1 = contamine50k1[:len(contamine50k2)]
    recovered50k1 = recovered50k1[:len(recovered50k2)]
    susceptible50k1 = susceptible50k1[:len(susceptible50k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine50k1['Nombre de contaminés'].values.tolist(),contamine50k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered50k1['Recovered'].values.tolist(),recovered50k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible50k1.values.tolist(),susceptible50k2.values.tolist())).mean())

### 100000 individus

In [ ]:
path100k1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population100000"
path100k2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population100000"

In [ ]:
temps = pd.read_csv(path100k1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

temps = pd.read_csv(path100k2 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
nombre_individus = 100000
temps_5000_mouvements = pd.read_csv(path100k1 + "/data_csv/X_mouvements_time.csv")
temps_5000_mouvements_mean = int(temps_5000_mouvements.mean(axis=0))
print("Temps moyen pour un individu de se déplacer 5000 fois : ", temps_5000_mouvements_mean, " [micro secondes]")
temps_total_5000_mouvements = int(temps_5000_mouvements_mean*nombre_individus*len(temps_5000_mouvements)/1000)
temps_total_run = int(temps['Run'].values)
ratio_mouvements_run = temps_total_5000_mouvements / temps_total_run
print("Pourcentage de temps passé à effectuer les mouvements sur le temps total du Run : ", ratio_mouvements_run*100, " [%]")

In [ ]:
contamine100k1 = pd.read_csv(path100k1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k1 = pd.read_csv(path100k1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine100k2 = pd.read_csv(path100k2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered100k2 = pd.read_csv(path100k2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 200

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine100k1, label='I_100000(t)')
I_MIX, = plt.plot(contamine100k2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered100k1, label='R_100000(t)')
R_MIX, = plt.plot(recovered100k2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

In [ ]:
iteration = 20

#5000
IX1 = pd.read_csv(path100k1 + "/data_csv/IX.csv", dtype=int)
IY1 = pd.read_csv(path100k1 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(15, 15))
vx1_Sain = [element for element in IX1.loc[iteration].values if element > 0]
vx1_Infected = [-element for element in IX1.loc[iteration].values if element < 0]
vy1_Sain = [element for element in IY1.loc[iteration].values if element > 0]
vy1_Infected = [-element for element in IY1.loc[iteration].values if element < 0]

#MIX
IX2 = pd.read_csv(path100k2 + "/data_csv/IX.csv", dtype=int)
IY2 = pd.read_csv(path100k2 + "/data_csv/IY.csv", dtype=int)
figure = plt.figure(figsize=(30, 15))
vx2_Sain = [element for element in IX2.loc[iteration].values if element > 0]
vx2_Infected = [-element for element in IX2.loc[iteration].values if element < 0]
vy2_Sain = [element for element in IY2.loc[iteration].values if element > 0]
vy2_Infected = [-element for element in IY2.loc[iteration].values if element < 0]

plt.subplot(1,2,1)
plt.title("SIR_référence, densité 1/16, 100000 individus, 5000 mouvements")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx1_Sain,vy1_Sain, color='green', label='Individus sains', s=3)
plt.scatter(vx1_Infected,vy1_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.subplot(1,2,2)
plt.title("SIR_référence, densité 1/16, 100000 individus, perfect mix")
plt.xlabel("X location")
plt.ylabel("Y location")
plt.scatter(vx2_Sain,vy2_Sain, color='green', label='Individus sains', s=3)
plt.scatter(vx2_Infected,vy2_Infected, color='red', label='Individus infectés')
plt.legend(loc=1)
plt.show()


### MSE

In [ ]:
susceptible100k1 = (N-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])
mse1 = SIR_mse_calc(N,contamine100k1,susceptible100k1,recovered100k1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible100k2 = (N-contamine100k2)['Nombre de contaminés'].subtract(recovered100k2['Recovered'])
mse2 = SIR_mse_calc(N,contamine100k2,susceptible100k2,recovered100k2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine100k1) < len(contamine100k2)):
    contamine100k2 = contamine100k2[:len(contamine100k1)]
    recovered100k2 = recovered100k2[:len(recovered100k1)]
    susceptible100k2 = susceptible100k2[:len(susceptible100k1)]
else:
    contamine100k1 = contamine100k1[:len(contamine100k2)]
    recovered100k1 = recovered100k1[:len(recovered100k2)]
    susceptible100k1 = susceptible100k1[:len(susceptible100k2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine100k1['Nombre de contaminés'].values.tolist(),contamine100k2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered100k1['Recovered'].values.tolist(),recovered100k2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible100k1.values.tolist(),susceptible100k2.values.tolist())).mean())

### 5000 mouvements comparaisons

In [ ]:
iterations = 100

figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_5000, = plt.plot(contamine5k1/5000, label='I_5000(t)')
I_20000, = plt.plot(contamine20k1/20000, label='I_20000(t)')
I_50000, = plt.plot(contamine50k1/50000, label='I_50000(t)')
I_100000, = plt.plot(contamine100k1/100000, label='I_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Infected normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[I_5000,I_20000,I_50000,I_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_5000, = plt.plot(recovered5k1/5000, label='R_5000(t)')
R_20000, = plt.plot(recovered20k1/20000, label='R_20000(t)')
R_50000, = plt.plot(recovered50k1/50000, label='R_50000(t)')
R_100000, = plt.plot(recovered100k1/100000, label='R_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Recovered normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[R_5000,R_20000,R_50000,R_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_5000, = plt.plot((5000-contamine5k1)['Nombre de contaminés'].subtract(recovered5k1['Recovered'])/5000, label='S_5000(t)')
S_20000, = plt.plot((20000-contamine20k1)['Nombre de contaminés'].subtract(recovered20k1['Recovered'])/20000, label='S_20000(t)')
S_50000, = plt.plot((50000-contamine50k1)['Nombre de contaminés'].subtract(recovered50k1['Recovered'])/50000, label='S_50000(t)')
S_100000, = plt.plot((100000-contamine100k1)['Nombre de contaminés'].subtract(recovered100k1['Recovered'])/100000, label='S_100000(t)')

plt.title("SIR_référence, densité 1/8, Compartiment Susceptible normalisé")
plt.xlabel("Itérations")
plt.ylabel("Individus normalisés")
plt.legend(handles=[S_5000,S_20000,S_50000,S_100000])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()